In [1]:
import requests
from bs4 import BeautifulSoup
import time
import os
import sys
import pandas as pd

sleep = 8

# map sentiment trend of hurt-comfort fics?

test = None

def chap_html_to_str(chap_html):
    #global test
    all_p = chap_html.find_all("p")
    #test = all_p
    str_arr = list(map(lambda x: str(x.string).replace("\xa0", ""), all_p))
    str_arr = [st for st in str_arr if st]
    return ' '.join(str_arr)

def scrape_fic(work_id, by_chapter = True):
    print("getting fic", work_id)
    url = 'http://archiveofourown.org/works/' + str(work_id) + "?view_full_work=true?view_adult=true"

    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    
    chaps_html = soup.find("div", id="chapters").find_all(class_="userstuff")
   
    chaps_clean = list(map(chap_html_to_str, chaps_html))
    time.sleep(sleep)
    
    return chaps_clean # if by_chapter else " ".join(chaps_clean)#, notes_clean, summ_html

[search url](https://archiveofourown.org/works?utf8=%E2%9C%93&work_search%5Bsort_column%5D=kudos_count&include_work_search%5Bfandom_ids%5D%5B%5D=65&work_search%5Bother_tag_names%5D=&work_search%5Bexcluded_tag_names%5D=&work_search%5Bcrossover%5D=F&work_search%5Bcomplete%5D=T&work_search%5Bwords_from%5D=&work_search%5Bwords_to%5D=&work_search%5Bdate_from%5D=&work_search%5Bdate_to%5D=&work_search%5Bquery%5D=&work_search%5Blanguage_id%5D=en&commit=Sort+and+Filter&tag_id=Avatar%3A+The+Last+Airbender"
)

In [3]:
search_url = "https://archiveofourown.org/works?utf8=%E2%9C%93&work_search%5Bsort_column%5D=kudos_count&include_work_search%5Bfandom_ids%5D%5B%5D=65&work_search%5Bother_tag_names%5D=&work_search%5Bexcluded_tag_names%5D=&work_search%5Bcrossover%5D=F&work_search%5Bcomplete%5D=T&work_search%5Bwords_from%5D=&work_search%5Bwords_to%5D=&work_search%5Bdate_from%5D=&work_search%5Bdate_to%5D=&work_search%5Bquery%5D=&work_search%5Blanguage_id%5D=en&commit=Sort+and+Filter&tag_id=Avatar%3A+The+Last+Airbender"

# single chapter only

test = None

def get_id_from_heading(heading_html):
    link = heading_html.find("a")
    if link:
        return link['href'].split("/")[2]
    
def parse_meta(meta_html):
    work_id = meta_html.get("id").split("_")[1]
    lang = meta_html.find_all(class_="language")[1].text
    words = int(meta_html.find_all(class_="words")[1].text.replace(",", ""))
    chapters = int(meta_html.find_all(class_="chapters")[1].text.split("/")[0])
    
    return work_id, lang, words, chapters

def get_ids(url, count, page=1, single_only=False):
    global test
    # what if we run out?
    
    url_page = url if page == 1 else url + "&page=" + str(page)
    
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    
    
    meta_list = soup.find_all(class_="blurb")
    
    meta_parsed = [parse_meta(meta_html) for meta_html in meta_list]

    #headings = soup.find_all("h4", class_="heading")
    
    #ids = list(map(get_id_from_heading, headings))
    #ids = [id for id in ids if id]
    
    ids = [x[0] for x in meta_parsed if x[1] == "English" and (True if not single_only else False or x[3] == 1)]
    
    if count < 20:
        return ids[:count]    
    return ids + get_ids(url, count-20, page+1)
    
ids = get_ids(search_url, 20, single_only=True)

# options: single chapter only, word count

In [4]:
ids

['14921627',
 '15753762',
 '14510448',
 '21526240',
 '19979299',
 '1078856',
 '1938096',
 '10990518',
 '21990778',
 '21086960',
 '19416352',
 '1874571',
 '15923450',
 '330627',
 '16964727',
 '24400723',
 '22011667']

In [145]:
texts = [scrape_fic(id) for id in ids]

getting fic 14921627
getting fic 15753762
getting fic 14510448
getting fic 21526240
getting fic 19979299
getting fic 1078856
getting fic 1938096
getting fic 10990518
getting fic 21990778
getting fic 21086960
getting fic 19416352
getting fic 1874571
getting fic 15923450
getting fic 330627
getting fic 16964727
getting fic 24400723
getting fic 22011667
